In [44]:
import re
import pandas as pd

import MeCab
import mojimoji

import numpy as np

import pprint

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

from pathlib import Path

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer

from gensim.models import KeyedVectors

## Load Data

In [4]:
class DataLoader:
    def __init__(self, text_path):
        path = Path(text_path)
        assert path.exists()
        assert path.is_dir()

        self.classes = list()
        self.dir = list()
        for p in path.iterdir():
            if p.is_dir():
                self.classes.append(p.name)
                self.dir.append(p)

        self.encode_dict = {k: i for i, k in enumerate(self.classes)}
        self.decode_dict = {i: k for i, k in enumerate(self.classes)}

        self.title = list()
        self.url = list()
        self.datetime = list()
        self.text = list()
        self.label = list()

        for p in self.dir:
            for d in p.iterdir():
                if d.name != "LICENSE.txt":
                    with open(d) as f:
                        split = f.readlines()
                    self.url.append(split[0].replace("\n", ""))
                    self.datetime.append(split[1].replace("\n", ""))
                    self.title.append(split[2].replace("\n", ""))
                    self.text.append(" ".join([
                        s.replace("\n", "").replace("\u3000", " ")
                        for s in split[3:]
                    ]))
                    self.label.append(self.encode_dict[p.name])
        self.data = pd.DataFrame({
            "date": self.datetime,
            "url": self.url,
            "title": self.title,
            "text": self.text,
            "label": self.label
        })

        self.tokenized = False

    def tokenize(self, tokenizer, kwargs={}):
        self.data["concatenated"] = self.data["title"] + " " + \
            self.data["text"]
        self.data["tokenized"] = self.data.concatenated.map(
            lambda x: tokenizer(x, **kwargs))
        self.tokenized = True

    def load(self):
        assert self.tokenized, "Needs tokenization before loading"
        train, test = train_test_split(
            self.data, test_size=0.3, shuffle=True, random_state=42)
        return train, test

In [24]:
loader = DataLoader(text_path="../input/text/")

In [26]:
loader.data.head()

,date,url,title,text,label
0,2011-10-30T10:15:00+0900,http://news.livedoor.com/article/detail/5978741/,【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か,2005年11月から翌2006年7月まで読売新聞にて連載された、直木賞作家・角田光代による...,0
1,2012-02-29T11:45:00+0900,http://news.livedoor.com/article/detail/6322901/,藤原竜也、中学生とともにロケット打ち上げに成功,「アンテナを張りながら生活をしていけばいい」 2月28日、映画『おかえり、はやぶさ』（...,0
2,2012-01-09T14:00:00+0900,http://news.livedoor.com/article/detail/6176324/,『戦火の馬』ロイヤル・プレミアにウィリアム王子＆キャサリン妃が出席,3月2日より全国ロードショーとなる、スティーブン・スピルバーグの待望の監督最新作『戦火の馬...,0
3,2012-05-19T12:00:00+0900,http://news.livedoor.com/article/detail/6573929/,香里奈、女子高生100人のガチンコ質問に回答「ラーメンも食べる」,女優の香里奈が18日、都内で行われた映画『ガール』（5月26日公開）の女子高生限定試写会に...,0
4,2011-10-05T19:11:00+0900,http://news.livedoor.com/article/detail/5914880/,ユージの前に立ちはだかったJOY「僕はAKBの高橋みなみを守る」,5日、東京・千代田区の内幸町ホールにて、映画『キャプテン・アメリカ/ザ・ファースト・アベン...,0


## Preprocess

In [15]:
tagger = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")

def nn_tokenizer(text):
    text = mojimoji.zen_to_han(text.replace("\n", ""), kana=False)
    text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
    parsed = tagger.parse(text).split("\n")
    parsed = [t.split("\t") for t in parsed]
    parsed = list(filter(lambda x: x[0] != "" and x[0] != "EOS", parsed))
    parsed = [p[2] for p in parsed]
    return parsed

In [16]:
loader.tokenize(nn_tokenizer)

data = loader.data

In [19]:
data.head()

,date,url,title,text,label,concatenated,tokenized
0,2011-10-30T10:15:00+0900,http://news.livedoor.com/article/detail/5978741/,【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か,2005年11月から翌2006年7月まで読売新聞にて連載された、直木賞作家・角田光代による...,0,【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か 2005年11月か...,"[【, DVD, エンター, !】, 誘拐犯, に, 育てる, られる, た, 女, が, ..."
1,2012-02-29T11:45:00+0900,http://news.livedoor.com/article/detail/6322901/,藤原竜也、中学生とともにロケット打ち上げに成功,「アンテナを張りながら生活をしていけばいい」 2月28日、映画『おかえり、はやぶさ』（...,0,藤原竜也、中学生とともにロケット打ち上げに成功 「アンテナを張りながら生活をしていけばいい...,"[藤原竜也, 、, 中学生, とともに, ロケット打ち上げ, に, 成功, 「, アンテナ,..."
2,2012-01-09T14:00:00+0900,http://news.livedoor.com/article/detail/6176324/,『戦火の馬』ロイヤル・プレミアにウィリアム王子＆キャサリン妃が出席,3月2日より全国ロードショーとなる、スティーブン・スピルバーグの待望の監督最新作『戦火の馬...,0,『戦火の馬』ロイヤル・プレミアにウィリアム王子＆キャサリン妃が出席 3月2日より全国ロード...,"[『, 戦火の馬, 』, ロイヤル, ・, プレミア, に, ウィリアム王子, &, キャサ..."
3,2012-05-19T12:00:00+0900,http://news.livedoor.com/article/detail/6573929/,香里奈、女子高生100人のガチンコ質問に回答「ラーメンも食べる」,女優の香里奈が18日、都内で行われた映画『ガール』（5月26日公開）の女子高生限定試写会に...,0,香里奈、女子高生100人のガチンコ質問に回答「ラーメンも食べる」 女優の香里奈が18日、都...,"[香里奈, 、, 女子高生, 100人, の, ガチンコ!, 質問, に, 回答, 「, ラ..."
4,2011-10-05T19:11:00+0900,http://news.livedoor.com/article/detail/5914880/,ユージの前に立ちはだかったJOY「僕はAKBの高橋みなみを守る」,5日、東京・千代田区の内幸町ホールにて、映画『キャプテン・アメリカ/ザ・ファースト・アベン...,0,ユージの前に立ちはだかったJOY「僕はAKBの高橋みなみを守る」 5日、東京・千代田区の内...,"[YU-G, の, 前, に, 立ちはだかる, た, JOY, 「, 僕, は, AKB, ..."


In [21]:
tk = Tokenizer(lower=True, filters="")
tk.fit_on_texts(data.tokenized.map(lambda x: " ".join(x)))

In [22]:
word_idx = tk.word_index
len(word_idx)

90866

## Check OOV

In [28]:
def load_embedding(path):
    binary = False
    if "bin" in path:
        binary = True
    emb = KeyedVectors.load_word2vec_format(path, binary=binary)
    return emb

In [29]:
stanby = load_embedding("/Users/arai301070/Data/stanby-jobs-200d-word2vector.bin")
wiki = load_embedding("/Users/arai301070/Data/model.vec")

stanby_words = set(stanby.vocab.keys())
wiki_words = set(wiki.vocab.keys())
livedoor_words = set(word_idx.keys())

In [30]:
oov_stanby = livedoor_words - stanby_words
oov_wiki = livedoor_words - wiki_words

print(len(oov_stanby))
print(len(oov_wiki))

48371
30477


In [33]:
in_stanby = oov_wiki - oov_stanby
in_wiki = oov_stanby - oov_wiki

print(len(in_stanby))
print(len(in_wiki))

4279
22173


## Transform

In [35]:
class VectorTransformer(nn.Module):
    def __init__(self, source_dim, target_dim):
        super(VectorTransformer, self).__init__()
        self.mat = nn.Linear(source_dim, target_dim)

    def forward(self, x):
        out = self.mat(x)
        return out

In [36]:
def create_loader(vocab, source_emb, target_emb):
    n_vec = len(vocab)
    source_dim = source_emb.vector_size
    target_dim = target_emb.vector_size

    x = np.zeros((n_vec, source_dim))
    y = np.zeros((n_vec, target_dim))
    for i, key in enumerate(vocab):
        source_vec = source_emb.get_vector(key)
        target_vec = target_emb.get_vector(key)
        x[i, :] = source_vec
        y[i, :] = target_vec
    x = torch.tensor(x, dtype=torch.float32).to("cpu")
    y = torch.tensor(y, dtype=torch.float32).to("cpu")
    dataset = data.TensorDataset(x, y)
    loader = data.DataLoader(dataset, batch_size=32, shuffle=True)
    return loader


intersection = stanby_words.intersection(wiki_words)
dataloader = create_loader(intersection, stanby, wiki)

In [37]:
model = VectorTransformer(stanby.vector_size, wiki.vector_size)
model.to("cpu")
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()

for _ in range(3):
    model.train()
    avg_loss = 0.
    for (x_batch, y_batch) in dataloader:
        y_pred = model(x_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(dataloader)

In [38]:
stanby_only_words = stanby_words - intersection
for _ in range(10):
    word = stanby_only_words.pop()
    emb = stanby.get_vector(word)
    tensor = torch.tensor(emb, dtype=torch.float32).to("cpu")
    pred = model(tensor).detach().numpy()
    similar = wiki.similar_by_vector(pred)
    print(f"word: {word}")
    print(f"similar: {similar}")

word: 
similar: [('sanjeev', 0.6729212999343872), ('Ekam', 0.6711547374725342), ('kushitani', 0.6677302122116089), ('Yukam', 0.6665024757385254), ('ameblo', 0.6538156270980835), ('belong', 0.653087317943573), ('Maxpoint', 0.6517963409423828), ('Tsuchiya', 0.6465193033218384), ('someone', 0.6425241231918335), ('Com.A', 0.6418524384498596)]
word: billings
similar: [('typically', 0.8285849690437317), ('exploitation', 0.8231680393218994), ('providing', 0.8215106725692749), ('accordance', 0.8210566639900208), ('mainly', 0.8163630366325378), ('therefore', 0.8141612410545349), ('Ekam', 0.8065015077590942), ('declared', 0.8038038015365601), ('determined', 0.80332350730896), ('suffer', 0.8029685020446777)]
word: ユーザエクスペリエンスデザイン
similar: [('モデル駆動型アーキテクチャ', 0.7261942625045776), ('xUnit', 0.725232720375061), ('ダイナミックリンクライブラリ', 0.7216801643371582), ('Standardization', 0.7208871841430664), ('ParaView', 0.7192454934120178), ('モデル変換言語', 0.7188775539398193), ('ドメインコントローラ', 0.718255877494812), ('WSDL', 

In [57]:
def find_similar_words(word):
    emb = stanby.get_vector(word)
    tensor = torch.tensor(emb, dtype=torch.float32).to("cpu")
    pred = model(tensor).detach().numpy()
    similar = wiki.similar_by_vector(pred)
    pprint.pprint(f"word: {word}")
    pprint.pprint(f"similar: {similar}")

In [58]:
find_similar_words("マイクロソフトソリューション")

'word: マイクロソフトソリューション'
("similar: [('しんきん北陸トライアングルネットワークATMサービス', 0.6291667222976685), "
 "('ダイナミックリンクライブラリ', 0.6054505109786987), ('モデル変換言語', 0.6053767800331116), "
 "('xUnit', 0.5980054140090942), ('シングルサインオン', 0.5928266048431396), "
 "('モデル駆動型アーキテクチャ', 0.5893920660018921), ('日開野町', 0.583366870880127), "
 "('ツールチェーン', 0.5811569094657898), ('ビジネスロジック', 0.5804120302200317), "
 "('ポリゴンメッシュ', 0.580378532409668)]")


In [48]:
find_similar_words("ユーザエクスペリエンスデザイン")

'word: ユーザエクスペリエンスデザイン'
("similar: [('モデル駆動型アーキテクチャ', 0.7261942625045776), ('xUnit', "
 "0.725232720375061), ('ダイナミックリンクライブラリ', 0.7216801643371582), "
 "('Standardization', 0.7208871841430664), ('ParaView', 0.7192454934120178), "
 "('モデル変換言語', 0.7188775539398193), ('ドメインコントローラ', 0.718255877494812), ('WSDL', "
 "0.7180397510528564), ('リッチインターネットアプリケーション', 0.7162258625030518), "
 "('CardSpace', 0.7149497866630554)]")


In [49]:
find_similar_words("スポットクーラー")

'word: スポットクーラー'
("similar: [('人感センサ', 0.6459254026412964), ('室外機', 0.6403393149375916), "
 "('冷暖房', 0.6274213790893555), ('給湯設備', 0.6240695714950562), ('換気扇', "
 "0.6236535906791687), ('空調', 0.6209656000137329), ('パワーコンディショナー', "
 "0.6202148795127869), ('乗務員宿泊所', 0.6122036576271057), ('ダウンライト', "
 "0.6091240644454956), ('車軸発電機', 0.6078335642814636)]")


In [51]:
find_similar_words("billings")

'word: billings'
("similar: [('typically', 0.8285849690437317), ('exploitation', "
 "0.8231680393218994), ('providing', 0.8215106725692749), ('accordance', "
 "0.8210566639900208), ('mainly', 0.8163630366325378), ('therefore', "
 "0.8141612410545349), ('Ekam', 0.8065015077590942), ('declared', "
 "0.8038038015365601), ('determined', 0.80332350730896), ('suffer', "
 '0.8029685020446777)]')


In [52]:
find_similar_words("ファーマライン")

'word: ファーマライン'
("similar: [('三菱東京フィナンシャル・グループ', 0.6641286611557007), ('ベインキャピタル', "
 "0.6620704531669617), ('プリンシパル・インベストメンツ', 0.6582026481628418), ('大和証券SMBC', "
 "0.6458470225334167), ('アリコジャパン', 0.6447579860687256), ('セントラルファイナンス', "
 "0.6438466906547546), ('アットローン', 0.6431422233581543), ('サーベラス・キャピタル・マネジメント', "
 "0.6415351033210754), ('リップルウッド・ホールディングス', 0.6388949155807495), "
 "('キヤノンマーケティングジャパン', 0.6377590894699097)]")


In [53]:
find_similar_words("選ら")

'word: 選ら'
("similar: [('ぎふしりつ', 0.645099401473999), ('かみあまくさ', 0.6444453597068787), "
 "('ちばしり', 0.6313422918319702), ('くちよ', 0.6203233003616333), ('あさひまち', "
 "0.6101150512695312), ('むすめどうじょうじ', 0.6086586713790894), ('東京都中央区佃', "
 "0.608351469039917), ('にししん', 0.6080089807510376), ('なごやせん', "
 "0.6051886081695557), ('ろちゅう', 0.6039457321166992)]")


In [54]:
def create_random_loader(vocab, source_emb, target_emb, sample_size=0.6):
    n_vec = int(sample_size * len(vocab))
    sample_vocab = np.random.choice(list(vocab), 
                                    size=n_vec, 
                                    replace=False)
    source_dim = source_emb.vector_size
    target_dim = target_emb.vector_size

    x = np.zeros((n_vec, source_dim))
    y = np.zeros((n_vec, target_dim))
    for i, key in enumerate(sample_vocab):
        source_vec = source_emb.get_vector(key)
        target_vec = target_emb.get_vector(key)
        x[i, :] = source_vec
        y[i, :] = target_vec
    x = torch.tensor(x, dtype=torch.float32).to("cpu")
    y = torch.tensor(y, dtype=torch.float32).to("cpu")
    dataset = data.TensorDataset(x, y)
    loader = data.DataLoader(dataset, batch_size=32, shuffle=True)
    return loader

In [59]:
class RandomVectorTransformer:
    def __init__(self, output_dim):
        self.output_dim = output_dim
        self.models = list()
        
    def fit(self, dataloaders):
        for dataloader in dataloaders:
            model = VectorTransformer(stanby.vector_size, wiki.vector_size)
            model.to("cpu")
            optimizer = optim.Adam(model.parameters())
            loss_fn = nn.MSELoss()

            for _ in range(3):
                model.train()
                avg_loss = 0.
                for (x_batch, y_batch) in dataloader:
                    y_pred = model(x_batch)
                    loss = loss_fn(y_pred, y_batch)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    avg_loss += loss.item() / len(dataloader)
                print(f"avg_loss: {avg_loss:.3f}")
            self.models.append(model)
            
    def predict(self, x):
        out = np.zeros(self.output_dim)
        n_estimators = len(self.models)
        for model in self.models:
            model.eval()
            out += model(x).detach().numpy() / n_estimators
        return out

In [63]:
loaders = []
np.random.seed(10)
for _ in range(10):
    ld = create_random_loader(intersection, stanby, wiki)
    loaders.append(ld)

In [64]:
rvt = RandomVectorTransformer(300)

In [65]:
rvt.fit(loaders)

avg_loss: 0.071
avg_loss: 0.068
avg_loss: 0.068
avg_loss: 0.071
avg_loss: 0.068
avg_loss: 0.068
avg_loss: 0.071
avg_loss: 0.068
avg_loss: 0.068
avg_loss: 0.071
avg_loss: 0.068
avg_loss: 0.068
avg_loss: 0.071
avg_loss: 0.068
avg_loss: 0.068
avg_loss: 0.071
avg_loss: 0.068
avg_loss: 0.068
avg_loss: 0.071
avg_loss: 0.068
avg_loss: 0.068
avg_loss: 0.071
avg_loss: 0.068
avg_loss: 0.068
avg_loss: 0.071
avg_loss: 0.068
avg_loss: 0.068
avg_loss: 0.070
avg_loss: 0.068
avg_loss: 0.068


In [68]:
def find_similar_words_ensemble(word):
    emb = stanby.get_vector(word)
    tensor = torch.tensor(emb, dtype=torch.float32).to("cpu")
    pred = rvt.predict(tensor)
    similar = wiki.similar_by_vector(pred)
    pprint.pprint(f"word: {word}")
    pprint.pprint(f"similar: {similar}")

In [70]:
find_similar_words_ensemble("ファーマライン")

'word: ファーマライン'
("similar: [('プリンシパル・インベストメンツ', 0.7054440379142761), ('ベインキャピタル', "
 "0.7009447813034058), ('日立キャピタル', 0.6900869011878967), ('キヤノンマーケティングジャパン', "
 "0.6856412291526794), ('三菱東京フィナンシャル・グループ', 0.6843117475509644), "
 "('リップルウッド・ホールディングス', 0.6840929985046387), ('アットローン', 0.6791737079620361), "
 "('サーベラス・キャピタル・マネジメント', 0.6786119937896729), ('アリコジャパン', 0.6736608743667603), "
 "('トヨタファイナンス', 0.6711728572845459)]")


In [71]:
find_similar_words_ensemble("スポットクーラー")

'word: スポットクーラー'
("similar: [('室外機', 0.6489385962486267), ('人感センサ', 0.642612099647522), ('冷暖房', "
 "0.636821448802948), ('パワーコンディショナー', 0.6355054378509521), ('空調', "
 "0.6311885118484497), ('エアコン', 0.6304082870483398), ('車軸発電機', "
 "0.6249876022338867), ('ダウンライト', 0.6181219816207886), ('センタースタンド', "
 "0.6160700917243958), ('受水槽', 0.6158645749092102)]")


In [73]:
find_similar_words_ensemble("マイクロソフトソリューション")

'word: マイクロソフトソリューション'
("similar: [('しんきん北陸トライアングルネットワークATMサービス', 0.6494717597961426), ('シングルサインオン', "
 "0.6058846116065979), ('xUnit', 0.5999319553375244), ('ダイナミックリンクライブラリ', "
 "0.5992327332496643), ('モデル変換言語', 0.5990589261054993), ('PaaS', "
 "0.5972484350204468), ('ノースゲートビルディング', 0.5944356918334961), ('OMLIS', "
 "0.5944176912307739), ('自動車ターミナル法', 0.594096839427948), "
 "('インターネット・プロトコル・スイート', 0.5927585959434509)]")


In [74]:
find_similar_words_ensemble("選ら")

'word: 選ら'
("similar: [('ぎふしりつ', 0.6515939831733704), ('かみあまくさ', 0.6432999968528748), "
 "('ちばしり', 0.6325711011886597), ('むすめどうじょうじ', 0.6321752071380615), ('あさひまち', "
 "0.6257833242416382), ('くちよ', 0.6246854066848755), ('ろちゅう', "
 "0.6073383092880249), ('ゅうけいほうそうしょ', 0.6062726974487305), ('ょうえい', "
 "0.6054717302322388), ('みしり', 0.6045638918876648)]")


In [75]:
find_similar_words_ensemble("ユーザエクスペリエンスデザイン")

'word: ユーザエクスペリエンスデザイン'
("similar: [('ダイナミックリンクライブラリ', 0.7336564064025879), ('xUnit', "
 "0.7298259139060974), ('リッチインターネットアプリケーション', 0.7288536429405212), "
 "('モデル駆動型アーキテクチャ', 0.7276886701583862), ('モデル変換言語', 0.7263566255569458), "
 "('Standardization', 0.7242797613143921), ('ParaView', 0.7225717902183533), "
 "('Liferay', 0.7207211256027222), ('アプリケーションプログラミングインタフェース', "
 "0.7199896574020386), ('DirectWrite', 0.7195067405700684)]")
